In [2]:
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pdb

In [3]:
from easydict import EasyDict as edict
from matplotlib import pyplot as plt
from torchvision import transforms as trans
from PIL import Image
from pathlib import Path
from models.Yolo_model import Yolo_model, build_targets, yolo_loss
import numpy as np
# np.seterr(all='raise')
import torch
from torch import nn, optim
# import torch.nn.functional as F
from utils.vis_utils import *
from utils.box_utils import *
from utils.dataset_tools import *
from utils.utils import *
from tensorboardX import SummaryWriter
from tqdm import tqdm_notebook as tqdm
from imgaug import augmenters as iaa
# s
from torch.utils.data import DataLoader

In [4]:
conf = edict()

conf.coco_anchors = [[10, 13], [16, 30], [33, 23], [30, 61], [62, 45],
                     [59, 119], [116, 90], [156, 198], [373, 326]]
conf.train_path = Path('/home/f/nvme/coco2017/train2017/')
conf.train_anno_path = Path(
    '/home/f/nvme/coco2017/annotations/instances_train2017.json')
conf.val_path = Path('/home/f/nvme/coco2017/val2017/')
conf.val_anno_path = Path(
    '/home/f/nvme/coco2017/annotations/instances_val2017.json')
conf.log_path = Path('/home/f/learning/yolo/log')
conf.model_path = Path('/home/f/learning/yolo/model')
conf.save_path = Path('/home/f/learning/yolo/save')
conf.ids_path = 'data/ids.npy'

conf.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

maps,correct_id_2_class = get_id_maps(conf)

loading annotations into memory...
Done (t=9.44s)
creating index...
index created!


In [5]:
conf.num_anchors = 3
conf.batch_size = 16
conf.input_size = 416
conf.scales = [32,16,8]

conf.running_norm = 0.
# conf.gdclip = 3000.
conf.num_workers = 8
conf.batch_size = 16
conf.gdclip = None
conf.coord_scale = 2.
conf.noobject_scale = 0.5
conf.object_scale = 5
conf.class_scale = 5.
conf.ignore_thresh = 0.5
conf.evaluate_iou_threshold = 0.5
conf.predict_confidence_threshold = 0.5
conf.pred_nms_iou_threshold = 0.5
conf.object_only = True
conf.warm_up_img_num = 12800

model = Yolo_model(conf)
model.to(conf.device)
conf.mean = model.res50_pyramid.model.mean
conf.std = model.res50_pyramid.model.std

conf.mse_loss = nn.MSELoss(size_average=False)
conf.bce_loss = nn.BCEWithLogitsLoss(size_average=False)

conf.board_loss_every = 5
conf.evaluate_every = 5
conf.board_pred_image_every = 5
# conf.board_loss_every = len(train_loader) // 100
# conf.evaluate_every = len(train_loader) // 10
# conf.board_pred_image_every = len(train_loader) // 2
# conf.save_every = len(train_loader) // 2
# conf.board_grad_norm = len(train_loader) // 10

In [6]:
train_ds = Coco_dataset(conf,conf.train_path,conf.train_anno_path,maps)
train_loader = DataLoader(train_ds,batch_size=conf.batch_size,shuffle=True,collate_fn=coco_collate_fn,pin_memory=True,num_workers=conf.num_workers)
val_dataset = datasets.CocoDetection(conf.val_path, conf.val_anno_path)
val_dataset.maps = maps
conf.transform_test = trans.Compose([
    trans.Resize([conf.input_size,conf.input_size]),
    trans.ToTensor(),
    trans.Normalize(conf.mean, conf.std)
])
val_loader = Coco_loader(
    conf,
    val_dataset,
    conf.transform_test,
    batch_size=conf.batch_size,
    hflip=False,
    shuffle=False)

loading annotations into memory...
Done (t=9.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.31s)
creating index...
index created!


In [7]:
paras = [*model.parameters()][159:]

optimizer = optim.SGD(paras,lr=1e-5,momentum=0.9,weight_decay=1e-4)

In [8]:
yolo = Yolo(conf,model,train_loader,val_loader,None,optimizer)

In [ ]:
imgs,bboxes_group,labels_group = next(iter(train_loader))

In [ ]:
imgs = imgs.to(conf.device)
for i,label in enumerate(labels_group):
    labels_group[i] = label.to(conf.device)
for i,bboxes in enumerate(bboxes_group):
    bboxes_group[i] = bboxes.to(conf.device)

In [ ]:
preds = yolo.model(imgs)

In [ ]:
preds.loss_feats[0].shape,preds.loss_feats[1].shape,preds.loss_feats[2].shape

In [ ]:
preds.pred_bboxes_group[0].shape,preds.pred_bboxes_group[1].shape,preds.pred_bboxes_group[2].shape

In [ ]:
yolo.model.head.anchors_group[0],yolo.model.head.anchors_group[1],yolo.model.head.anchors_group[2]

In [9]:
imgs,labels_group,bboxes_group = next(iter(val_loader))

In [10]:
torch.no_grad

torch.autograd.grad_mode.no_grad

In [11]:
imgs = imgs.to(conf.device)
for i,label in enumerate(labels_group):
    labels_group[i] = label.to(conf.device)
for i,bboxes in enumerate(bboxes_group):
    bboxes_group[i] = bboxes.to(conf.device)

In [12]:
yolo.model.eval()
# yolo.model.train()
yolo.model.training

False

In [13]:
torch.cuda.empty_cache()

In [14]:
preds = yolo.model(imgs)

In [16]:
targets, gt_mask, conf_weight, coord_mask = build_targets(
                conf, preds.pred_bboxes_group, bboxes_group, labels_group,
                yolo.model.head.anchors_group, True)

In [17]:
targets[0][0,0,:,:,2]

tensor(1.00000e-02 *
       [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  7.6035,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  6.9783, 

In [15]:
preds.loss_feats[0][0,0,:,:,2]

tensor(1.00000e-05 *
       [[-0.2247,  1.6529,  0.6032, -0.0218, -0.2112,  0.8989,  0.4619,
          4.0221,  3.3634,  1.1091,  2.0642,  0.2622,  2.4415],
        [-0.4093, -0.1864, -0.5303, -0.6787, -1.1372, -0.8606, -0.2409,
         -0.1790, -0.0450,  0.1145, -0.0065, -0.1989,  1.5258],
        [-0.5671, -0.4566, -0.5687, -1.0066, -1.2815, -1.0193, -0.4439,
         -0.3597, -0.3292, -0.1940, -0.0012, -0.4530, -0.1263],
        [-0.5763, -0.4204, -0.6985, -0.9281, -1.3681, -1.1011, -0.9010,
         -0.7337, -1.0654, -0.8913, -0.4209, -0.5206, -0.1571],
        [-0.3178, -0.4637, -0.9853, -0.8628, -1.1132, -0.9855, -0.7026,
         -0.9720, -0.7557, -0.3746, -0.6876, -0.5944, -0.1568],
        [-0.3854, -0.2844, -0.4701, -0.4231, -0.2201, -0.3224, -0.3382,
         -0.9660, -0.8575, -0.5539, -0.7063, -0.6363,  0.9414],
        [-0.6017, -0.2274, -0.5819,  0.6758,  1.4849, -0.0612, -0.1687,
         -0.6444, -1.0186, -0.7836, -0.6156, -0.1500,  3.7273],
        [-0.4848, -0.5405, 

In [15]:
preds.loss_feats[0][0,0,:,:,2]

tensor([[ 0.1274,  0.1476,  0.0531,  0.0618,  0.1638,  0.0323, -0.0022,
         -0.0010,  0.0960,  0.0013, -0.0084,  0.0162,  0.0846],
        [ 0.0840, -0.0108, -0.0113, -0.0048,  0.1058,  0.0622,  0.0556,
         -0.0001,  0.1340, -0.0084,  0.0045,  0.0369,  0.1134],
        [ 0.0526,  0.0696, -0.0188, -0.0100,  0.0093, -0.0269, -0.0062,
          0.0656,  0.0862,  0.1236, -0.0148,  0.0566, -0.0083],
        [ 0.0528, -0.0076,  0.0203, -0.0051, -0.0133, -0.0058, -0.0091,
         -0.0087, -0.0005, -0.0156, -0.0180, -0.0270, -0.0048],
        [ 0.1384, -0.0087,  0.0508,  0.0442, -0.0071, -0.0067, -0.0004,
         -0.0036, -0.0132,  0.0217, -0.0264, -0.0212,  0.0021],
        [ 0.1000, -0.0032,  0.0548,  0.0964,  0.0267,  0.0465,  0.1323,
          0.1315,  0.0181,  0.0101,  0.2290,  0.0597,  0.0538],
        [ 0.0511,  0.0624,  0.0990,  0.0202, -0.0178,  0.0071, -0.0001,
          0.1563,  0.0213, -0.0111,  0.1055,  0.0141, -0.0005],
        [-0.0032,  0.0168, -0.0143,  0.0354, -0.

In [22]:
targets, gt_mask, conf_weight, coord_mask = build_targets(
                conf, preds.pred_bboxes_group, bboxes_group, labels_group,
                yolo.model.head.anchors_group, True)

In [23]:
targets[0][0,0,:,:,2]

tensor(1.00000e-02 *
       [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  7.6035,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  6.9783, 

In [24]:
losses = yolo_loss(conf, preds.loss_feats, targets, gt_mask,conf_weight, coord_mask)

In [25]:
losses

losses(loss_total=tensor(39224.7891, device='cuda:0'), loss_xy=30294.765625, loss_wh=6826.3720703125, loss_conf=1868.921142578125, loss_cls=234.73077392578125)

In [ ]:
yolo.seen = 0

In [ ]:
yolo.evaluate(conf,5,verbose=True)

In [1]:
from torchvision import datasets
from torchvision import transforms as trans
import pdb
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from utils.vis_utils import *
from utils.box_utils import *
from utils.dataset_tools import *
from utils.utils import *
from models.Yolo_head import Yolo_loss
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt


def calc_preds(conf, preds, object_only = True):
    bboxes_group = []
    labels_group = []
    nB = len(preds.loss_feats[0])
    with torch.no_grad():
        for nb in range(nB):
            bboxes_predicted = []
            cls_predicted = []
            conf_predicted = []
            for l in range(3):
                confidences = preds.loss_feats[l][nb][...,4]
                cls_conf_preds,classes = torch.max(preds.loss_feats[l][nb][...,5:],dim=-1)
                bboxes = preds.pred_bboxes_group[l][nb]
                if object_only:
                    final_conf = confidences
                else:
                    final_conf = confidences * cls_conf_preds
                predicted_mask = final_conf > conf.predict_confidence_threshold
                bboxes_predicted.append(bboxes[predicted_mask])
                cls_predicted.append(classes[predicted_mask])
                conf_predicted.append(final_conf[predicted_mask])
            bboxes_predicted = torch.cat(bboxes_predicted)
            cls_predicted = torch.cat(cls_predicted)
            conf_predicted = torch.cat(conf_predicted)
            if len(bboxes_predicted) != 0:
                picked_boxes = non_max_suppression(xcycwh_2_xywh(bboxes_predicted).cpu().numpy(),
                                                   conf_predicted.cpu().numpy(),
                                                   conf.pred_nms_iou_threshold)
                bboxes_group.append(bboxes_predicted[picked_boxes])
                labels_group.append(cls_predicted[picked_boxes])
            else:
                bboxes_group.append(torch.tensor([0.,0.,0.,0.]).unsqueeze(0).to(conf.device))
                labels_group.append(torch.tensor([0]).to(conf.device))             
    return bboxes_group,labels_group

class Yolo(object):
    def __init__(self,
                 conf,
                 model=None,
                 train_loader=None,
                 val_loader=None,
                 writer=None,
                 optimizer=None,
                 step=0,
                 seen=0):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.writer = writer
        self.step = step
        self.seen = seen
        self.optimizer = optimizer
        
    def save_state(self,conf,val_loss,extra=None):
        torch.save(self.model.state_dict(),conf.model_path/
                   ('{}_val_loss:{}_model_seen:{}_step:{}_{}.pth'.format(get_time(),val_loss,self.seen,self.step,extra)))
        torch.save(self.optimizer.state_dict(),conf.model_path/
                    ('{}_val_loss:{}_optimizer_seen:{}_step:{}_{}.pth'.format(get_time(),val_loss,self.seen,self.step,extra)))
        
    def predict(self,conf,imgs,object_only=True,return_img=False):
        imgs = imgs.to(conf.device)
        nB = len(imgs)
        self.model.eval()
        preds = self.model(imgs)
        bboxes_group,labels_group = calc_preds(conf, preds, object_only = conf.object_only)
        self.model.train()         
        if return_img:
            return show_util(conf,0,imgs,labels_group,bboxes_group,self.train_loader.dataset.maps[2])
        else:
            return bboxes_group,labels_group
    
    def evaluate(self, conf, batches=100 ,verbose=False):
        self.val_loader.current = 0
        self.model.eval()
        if verbose:
            loader = tqdm(iter(self.val_loader),total = batches)
        else:
            loader = iter(self.val_loader)
        
        running_loss = 0.
        running_loss_xy = 0.
        running_loss_wh = 0.
        running_loss_conf = 0.
        running_loss_cls = 0.
        n_correct = 0
        n_gt = 0
        n_pred = 0
        cls_correct_num = 0
        batch_count = 0    
        warm_up = self.seen < conf.warm_up_img_num
        with torch.no_grad():         
            for imgs,labels_group,bboxes_group in loader:   
                if batch_count < batches:
                    imgs = imgs.to(conf.device)
                    for i,label in enumerate(labels_group):
                        labels_group[i] = label.to(conf.device)
                    for i,bboxes in enumerate(bboxes_group):
                        bboxes_group[i] = bboxes.to(conf.device)
                    preds = self.model(imgs)

                    targets, gt_mask, conf_weight, coord_mask = build_targets(
                        conf, preds.pred_bboxes_group, bboxes_group, labels_group,
                        yolo.model.head.anchors_group, warm_up)
                    
                    pdb.set_trace()

                    losses = yolo_loss(conf, preds.loss_feats, targets, gt_mask,conf_weight, coord_mask)
                    running_loss += losses.loss_total.item()
                    running_loss_xy += losses.loss_xy
                    running_loss_wh += losses.loss_wh
                    running_loss_conf += losses.loss_conf
                    running_loss_cls += losses.loss_cls                    
                    
                    bboxes_group_pred,labels_group_pred = calc_preds(conf, preds, object_only = conf.object_only)

                    for nb in range(len(imgs)):
                        pred_bboxes = bboxes_group_pred[nb]
                        gt_bboxes = bboxes_group[nb]
                        ious = cal_ious_xcycwh(pred_bboxes, gt_bboxes).to(conf.device)
                        max_matched_iou_gt,max_matched_box_idx_gt = torch.max(ious,dim=1)
                        matched_mask = max_matched_iou_gt > conf.evaluate_iou_threshold
                        matched_classes = labels_group_pred[nb][matched_mask]
                        cls_correct_num += torch.sum(matched_classes == \
                                                     labels_group[nb][max_matched_box_idx_gt][matched_mask]).item()
                        n_pred += len(pred_bboxes)
                        n_gt += len(gt_bboxes)
                        n_correct += torch.sum(matched_mask).item()
                    batch_count += 1
                else:
                    break
                    
        precision = n_correct/n_gt
        recall = n_correct/n_pred
        f1 = 2*precision*recall / (precision + recall + 1e-8)
        cls_acc = cls_correct_num/n_gt
        
        self.model.train()
        return running_loss/batches,\
                running_loss_xy/batches,\
                running_loss_wh/batches,\
                running_loss_conf/batches,\
                running_loss_cls/batches,\
                precision,recall,f1,cls_acc        
    
    def find_lr(self,conf,init_value = 1e-8, final_value=10., beta = 0.98,num = None):
        if not num:
            num = len(self.train_loader) // 5
        mult = (final_value / init_value) ** (1/num)
        lr = init_value
        self.optimizer.param_groups[0]['lr'] = lr
        avg_loss = 0.
        best_loss = 0.
        batch_num = 0
        losses = []
        log_lrs = []
        for imgs,labels_group,bboxes_group in tqdm(iter(self.train_loader),total=num):
            batch_num += 1
            imgs = imgs.to(conf.device)
            for i,label in enumerate(labels_group):
                labels_group[i] = label.to(conf.device)
            for i,bboxes in enumerate(bboxes_group):
                bboxes_group[i] = bboxes.to(conf.device)            
                
            self.optimizer.zero_grad()
            
            preds = self.model(imgs)

            targets, gt_mask, conf_weight, coord_mask = build_targets(
                conf, preds.pred_bboxes_group, bboxes_group, labels_group,
                yolo.model.head.anchors_group, warm_up)

            losses = yolo_loss(conf, preds.loss_feats, targets, gt_mask,conf_weight, coord_mask)

            #Compute the smoothed loss
            avg_loss = beta * avg_loss + (1-beta) *losses.loss_total.item()
            self.writer.add_scalar('avg_loss',avg_loss,batch_num)
            smoothed_loss = avg_loss / (1 - beta**batch_num)
            self.writer.add_scalar('smoothed_loss',smoothed_loss,batch_num)
            #Stop if the loss is exploding
            if batch_num > 1 and smoothed_loss > 10 * best_loss:
                print('exited with best_loss at {}'.format(best_loss))
                plt.plot(log_lrs[10:-5],losses[10:-5])
                return log_lrs, losses
            #Record the best loss
            if smoothed_loss < best_loss or batch_num==1:
                best_loss = smoothed_loss
            #Store the values
            losses.append(smoothed_loss)
            log_lrs.append(math.log10(lr))
            self.writer.add_scalar('log_lr',math.log10(lr),batch_num)
            #Do the SGD step
            losses.loss_total.backward()
            self.optimizer.step()
            #Update the lr for the next step
            lr *= mult
            self.optimizer.param_groups[0]['lr'] = lr
            if batch_num > num:
                return log_lrs, losses 
    
    def train(self, conf, epochs, log):
        running_loss = 0.
        running_loss_xy = 0.
        running_loss_wh = 0.
        running_loss_conf = 0.
        running_loss_cls = 0.

        for e in range(epochs):
            self.train_loader.current = 0
            for imgs, labels_group, bboxes_group in tqdm(iter(self.train_loader)):

                warm_up = True if self.seen < 12800 else False

                imgs = imgs.to(conf.device)
                for i, label in enumerate(labels_group):
                    labels_group[i] = label.to(conf.device)
                for i, bboxes in enumerate(bboxes_group):
                    bboxes_group[i] = bboxes.to(conf.device)

                self.optimizer.zero_grad()

                preds = self.model(imgs)

                targets, gt_mask, conf_weight, coord_mask = build_targets(
                    conf, preds.pred_bboxes_group, bboxes_group, labels_group,
                    yolo.model.head.anchors_group, warm_up)

                losses = yolo_loss(conf, preds.loss_feats, targets, gt_mask,conf_weight, coord_mask)

                losses.loss_total.backward()

                if conf.gdclip:
                    clip_grad_norm_log_(
                        conf, self.optimizer.param_groups[0]['params'],
                        conf.gdclip, self.writer, self.step)

                self.optimizer.step()
                self.step += 1
                self.seen += len(imgs)

                running_loss += losses.loss_total.item()
                running_loss_xy += losses.loss_xy
                running_loss_wh += losses.loss_wh
                running_loss_conf += losses.loss_conf
                running_loss_cls += losses.loss_cls

                if self.step % conf.board_loss_every == 0:
                    if warm_up:
                        self.writer.add_scalar('loss_warm_up',running_loss / conf.board_loss_every, self.step)
                        self.writer.add_scalar('loss_xy_warm_up',running_loss_xy / conf.board_loss_every, self.step)
                        self.writer.add_scalar('loss_wh_warm_up',running_loss_wh / conf.board_loss_every, self.step)
                        self.writer.add_scalar('loss_conf_warm_up',running_loss_conf / conf.board_loss_every,self.step)
                        self.writer.add_scalar('loss_cls_warm_up',running_loss_cls / conf.board_loss_every,self.step)
                    else:
                        self.writer.add_scalar('loss', running_loss / conf.board_loss_every,self.step)
                        self.writer.add_scalar('loss_xy', running_loss_xy / conf.board_loss_every,self.step)
                        self.writer.add_scalar('loss_wh', running_loss_wh / conf.board_loss_every,self.step)
                        self.writer.add_scalar('loss_conf',running_loss_conf / conf.board_loss_every,self.step)
                        self.writer.add_scalar('loss_cls',running_loss_cls / conf.board_loss_every,self.step)

                    running_loss = 0.
                    running_loss_xy = 0.
                    running_loss_wh = 0.
                    running_loss_conf = 0.
                    running_loss_cls = 0.
                
                if self.step % conf.evaluate_every == 0:
                    val_loss,\
                    val_loss_xy,\
                    val_loss_wh,\
                    val_loss_conf,\
                    val_loss_cls,\
                    precision,recall,f1,cls_acc = self.evaluate(conf,100,50)

                    self.writer.add_scalar('val_loss',val_loss,self.step)
                    self.writer.add_scalar('val_loss_xy',val_loss_xy,self.step)
                    self.writer.add_scalar('val_loss_wh',val_loss_wh,self.step)
                    self.writer.add_scalar('val_loss_conf',val_loss_conf,self.step)
                    self.writer.add_scalar('val_loss_cls',val_loss_cls,self.step)
                    self.writer.add_scalar('val_precision',precision,self.step)
                    self.writer.add_scalar('val_recall',recall,self.step)
                    self.writer.add_scalar('val_f1',f1,self.step)
                    self.writer.add_scalar('val_cls_acc',cls_acc,self.step)    

                if self.step % conf.board_pred_image_every == 0:
                    for i in range(20):
                        img,_ = self.val_loader.dataset[i]
                        img_tensor = self.predict(self.val_loader.transform(img),
                                                  conf,
                                                  conf.predict_confidence_threshold,
                                                  only_objectness=True,
                                                  return_img=True)
                        self.writer.add_image('pred_image_{}'.format(i),img_tensor,global_step=self.step)

                if self.step % conf.save_every == 0:
                    self.save_state(conf,val_loss,extra=log)

In [ ]:
targets, gt_mask, conf_weight, coord_mask = build_targets(
    conf,
    preds.pred_bboxes_group,
    bboxes_group,
    labels_group,
    yolo.model.head.anchors_group,
    warm_up=True)

In [ ]:
losses = yolo_loss(conf,preds.loss_feats,targets,gt_mask,conf_weight,coord_mask)

In [ ]:
from torch import nn

In [ ]:
losses.